# Making Quantum States

In this tutorial, we use the quantum circuits and operations we learned in the previous tutorials and get aquainted with the different kinds of quantum states we can prepare in PennyLane. For a full list of quantum functions available for state preparation, see [qubit-state-prep](https://pennylane.readthedocs.io/en/latest/code/ops/qubit.html#state-preparation) and [cv-state-prep](https://pennylane.readthedocs.io/en/latest/code/ops/cv.html#state-preparation)

In [1]:
# first we import the essentials

import pennylane as qml
from pennylane import numpy as np

### Qubit state Preparation

Lets first look at the features PennyLane provides for state preparation on the `default.qubit` device.  

> [BasisState](https://pennylane.readthedocs.io/en/latest/code/ops/qubit.html#pennylane.ops.qubit.BasisState) function prepares the device in a single computational basis state. For example, if we are working with two qubits and want to initialize  the device in the state $\mid10\rangle = \begin{pmatrix} 0 \\ 0 \\ 1 \\ 0 \end{pmatrix}$, we can use `BasisState` function. We will need to pass this state as an array, i.e `np.array([1,0])`. If we are working with three qubits and want to initialize the device in the state $\mid111\rangle = \begin{pmatrix} 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 1 \end{pmatrix}$, we will need to pass this desired state in the form of an array, i.e `np.array([1,1,1])`

In [2]:
# lets look at an example

dev1 = qml.device('default.qubit', wires=2)

@qml.qnode(dev1)
def make_State1(n=None):
    qml.BasisState(n, wires=[0,1])
    qml.CNOT(wires=[0,1])
    return qml.expval.PauliZ(0), qml.expval.PauliZ(1)

In [3]:
nstr = np.array([1,0])
make_State1(n=nstr)

array([-1., -1.])

This output makes sense as applying the CNOT gate transforms $\mid10\rangle$ to $\mid11\rangle$ state. 

> [QubitStateVector](https://pennylane.readthedocs.io/en/latest/code/ops/qubit.html#pennylane.ops.qubit.QubitStateVector) function prepares the device subsystems by using the given ket vector in the Fock basis. This acts similar to the `BasisState` function except:
- It can be used to prepare more complicated wavefunctions
- we need to provide the whole ket vector explicitly. For example, to prepare the state $\mid10\rangle$, we explicitly give the ket state $\begin{pmatrix} 0 \\ 0 \\ 1 \\ 0 \end{pmatrix}$, i.e `np.array([0,0,1,0])` 

In [4]:
dev2 = qml.device('default.qubit', wires=2)

@qml.qnode(dev2)
def make_State2(s=None):
    qml.QubitStateVector(s, wires=[0,1])
    return qml.expval.PauliZ(0),qml.expval.PauliZ(1)

Lets use this function to make the entangled state $|\psi\rangle = \frac{1}{\sqrt{2}}(|00\rangle-i|11\rangle)$

In [5]:
state = np.array([1+0.j, 0.j, 0.j, 0-1.j])*np.sqrt(2)
print(state)

make_State2(s=state)

[1.41421356+0.j         0.        +0.j         0.        +0.j
 0.        -1.41421356j]


array([0., 0.])

As we make local $\langle\hat{\sigma}_z\rangle$ measurements on the reduced states of $|\psi\rangle$, each subsystem is completely mixed and local expectation values average to zero.

### Gaussian State Preparation

Lets look at the features PennyLane provides for state preparation on the `default.gaussian` device.  

> [CoherentState](https://pennylane.readthedocs.io/en/latest/code/ops/cv.html#pennylane.ops.cv.CoherentState) function prepares a coherent state with the given displacement magnitude $\mid\alpha\mid$ and angle $\phi$

In [6]:
dev3 = qml.device('default.gaussian', wires=1)

@qml.qnode(dev3)
def make_State3(alpha_mag, phi):
    qml.CoherentState(alpha_mag, phi, wires=0)
    return qml.expval.MeanPhoton(0)

In [7]:
make_State3(2,0)

4.0

This makes sense as $\langle\hat{n}\rangle = \mid\alpha\mid^{2}$

> [DisplacedSqueezedState](https://pennylane.readthedocs.io/en/latest/code/ops/cv.html#pennylane.ops.cv.DisplacedSqueezedState) function prepares a displaced squeezed state in the phase space by applying a displacement operator followed by a squeezing operator - $D(\alpha)S(z)\mid0\rangle$ - like we saw in an example in the last tutorial on Quantum operators.  

> [GaussianState](https://pennylane.readthedocs.io/en/latest/code/ops/cv.html#pennylane.ops.cv.GaussianState) function prepares a Gaussian state on the whole system with the given parameters. It requires two input arguments: 
- a concatenated vector of the mean Position and Momentum values of all modes. For example, to prepare two Gaussian modes, we need to input the vector $[\langle x\rangle_0,\langle x\rangle_1,\langle p\rangle_0,\langle p\rangle_1]$
- the covariance matrix


In [8]:
dev4 = qml.device('default.gaussian', wires=2)

@qml.qnode(dev4)
def make_State4(r,V):
    qml.GaussianState(r,V,wires=[0,1])
    return qml.expval.MeanPhoton(0), qml.expval.MeanPhoton(1)

In [9]:
mean_array = np.array([2,2,0,0])
Cov_mat = np.eye(4)

make_State4(mean_array,Cov_mat)

array([1., 1.])

<font color=brown>**NOTE:**</font> We used $\langle x\rangle=2$ <font color=brown>and</font> $\langle p\rangle=0$. As `MeanPhoton` returns the expectation value of the number operator $\langle \hat{n} \rangle$, we have to use x-representation of `Coherent states` to calculate the mean Position and Momentum:
$$\langle x\rangle = \sqrt{\frac{2\hbar}{mw}} \Re(\alpha)$$
$$\langle p\rangle = \sqrt{2\hbar mw} \Im(\alpha)$$

<font color=brown>**NOTE:** In PennyLane, we assume that $\hbar = 2$ and factor $mw=1$</font>